In [ ]:
from math import exp
from sklearn.preprocessing import OneHotEncoder
from typing import List
from tqdm.notebook import tqdm
import seaborn as sns
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import dill

from sklearn.datasets import fetch_openml
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state

# Untuk downscale gambar, maaf kak, laptop kentang kami menderita karena ukuran 28x28 
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
import tensorflow as tf

In [ ]:
class Neuron:
    def __init__(self, value, _children=(), _op='', label=''):
        self.value = value
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label # for visualization

    def __repr__(self):
        stringVal = f"{self.value}"
        return stringVal

    def __add__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value + other.value, (self, other), '+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)

    def __mul__(self,other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        out = Neuron(self.value * other.value, (self, other), '*')
        def _backward():
            self.grad += other.value * out.grad
            other.grad += self.value * out.grad
        out._backward = _backward
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers"
        out = Neuron(self.value**other, (self, ), f'**{other}')
        def _backward():
            self.grad += other * (self.value ** (other - 1)) * out.grad
        out._backward = _backward
        return out

    def __truediv__(self, other):
        other = other if isinstance(other, Neuron) else Neuron(other)
        return self * other**-1

    def exp(self, pos=1):
        x = self.value*pos
        out = Neuron(exp(x), (self, ), 'exp')
        def _backward():
            self.grad += out.value * out.grad * pos
        out._backward = _backward
        return out

    def log(self):
        x = self.value
        out = Neuron(np.log(x), (self,), 'log')
        def _backward():
            self.grad += (1/x) * out.grad
        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other

    def __rsub__(self, other): # other - self
        return -(self - other)

    def __rmul__(self, other): # other * self
        return self * other

    def __rtruediv__(self, other): # other / self
        other = other if isinstance(other, Neuron) else Neuron(other)
        return other * self**-1

    def __lt__(self,other):
        if(isinstance(other,Neuron)):
            return self.value < other.value
        else:
            return self.value < other

    def __gt__(self,other):
        if(isinstance(other,Neuron)):
            return self.value > other.value
        else:
            return self.value > other

    def __le__(self,other):
        if(isinstance(other,Neuron)):
            return self.value <= other.value
        else:
            return self.value <= other

    def __ge__(self,other):
        if(isinstance(other,Neuron)):
            return self.value >= other.value
        else:
            return self.value >= other


    def backward(self):
        topology = []
        visited = set()
        def build_topology(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topology(child)
                topology.append(v)
        build_topology(self)

        self.grad = 1.0
        for node in reversed(topology):
            node._backward()

    def __float__(self):
        return float(self.value)

    def __format__(self, other):
        return format(float(self), other)

    def __round__(self,other):
        return round(self.value,other)

    def getGrad(self):
        return float(self.grad)

    def hello(self):
        print("hello")

In [ ]:
# Wrapper, propose better name?
# Maybe MLP? (multi layer perceptron, since it is a wrapper for mutliple layers of neurons)
class LayerWrapper:
    valid_loss_func = {"mse","bce","cce"}

    def __init__(self,
                 layer_neuron_width:List[int],
                 activation_function:str = "linear",
                 weight_initialization:str = "uniform",
                 loss_function:str = "mse",
                 uniform_parameter: tuple = (0, 10, 0),
                 normal_parameter: tuple = (5, 3, 0),
                 batch_size: int = 1,
                 learning_rate: float = 0.1,
                 max_epoch: int = 1,
                 verbose: int = 0,
                 regularization_type: str = "Unregularized",
                 regularization_lambda: float = 0
                 ):

        self.learning_rate = learning_rate
        self.loss_function = loss_function
        self.layer_selector:List[Layers] = []
        self.layer_count = len(layer_neuron_width)
        self.loss = Neuron(0)
        self.error_acc = Neuron(0)
        self.batch_size = batch_size
        self.max_epoch = max_epoch
        self.verbose = verbose
        self.regularization_type: str = regularization_type,
        self.regularization_lambda: float = regularization_lambda
        Layers(
            wrapper=self,
            layer_neuron_width=layer_neuron_width,
            activation_function=activation_function,
            weight_initialization=weight_initialization,
            uniform_parameter=uniform_parameter,
            normal_parameter=normal_parameter
        )

        self.train_class_value = LayerWrapper.neuronGenerator(np.zeros,(len(self.layer_selector[-1]._neurons)))
        self.validation_class_value = LayerWrapper.neuronGenerator(np.zeros,(len(self.layer_selector[-1]._neurons)))

    def verboseprint(self, *args: str, nl: bool=True):
        print(*args, end='\n' if nl else '	') if self.verbose else None

    def save(self,filename):
        with open(f"{filename}","wb") as file:
            dill.dump(self,file)

    def load(self,filename):
        with open(f"{filename}","rb") as file:
            self = dill.load(file)

    # Function, dengan retval
    @staticmethod
    def neuronGenerator(generator, *args, **kwargs):
        value = generator(*args, **kwargs)
        return np.vectorize(Neuron)(value)

    def loss_func(self,Y_pred,Y_target) -> float:
        if self.loss_function == "mse":
            return LayerWrapper.meanSquareError(Y_pred,Y_target)
        elif self.loss_function == "bce":
            return LayerWrapper.binaryCrossEntropy(Y_pred,Y_target)
        elif self.loss_function == "cce":
            return LayerWrapper.categoricalCrossEntropy(Y_pred,Y_target)

    @staticmethod
    def meanSquareError(Y_pred,Y_target) -> float:
        tempPredictClass = Y_pred
        tempTrueClass = Y_target
        subtracted = np.subtract(tempTrueClass,tempPredictClass)
        squared = np.square(subtracted)
        return np.sum(squared)/len(tempTrueClass)

    @staticmethod
    def binaryCrossEntropy(Y_pred: "Layers",Y_target) -> float:
        tempPredictClass = Y_pred
        tempTrueClass = Y_target
        logFunc = np.vectorize(lambda n: n.log())
        oneMinTarget = np.subtract(1, Y_target)
        # print(tempPredictClass)
        oneMinPredict = np.subtract(1, tempPredictClass)
        logPredictedClass = logFunc(tempPredictClass)
        logOneMinPredict = logFunc(oneMinPredict)
        return -np.sum(np.add(tempTrueClass*logPredictedClass, oneMinTarget*logOneMinPredict))/len(tempTrueClass)

    @staticmethod
    def categoricalCrossEntropy(Y_pred: "Layers",Y_target) -> float:
        # Categorical Cross Entropy target will be an array cause the target will be encoded
        # For example if we have 3 classes 0, 1, 2 and the target for this instance is 1, then the target will be [0, 1, 0]
        # The predict or the output we get must be probability for every classes in target
        # This error formula only can be used with softmax activation function
        encoder = OneHotEncoder(sparse_output=False)
        tempTrueClass = encoder.fit_transform(np.array(Y_target).reshape(-1, 1))
        tempPredictClass = encoder.fit_transform(np.array(Y_pred).reshape(-1, 1))
        # tempPredictClass = Y_pred   # ingat ini array of list yh, nvm better gua encode di dalem sini aja
        # tempTrueClass = Y_target    # ini juga yh
        logFunc = np.vectorize(lambda n: n.log())
        logPredictedClass = logFunc(tempPredictClass)
        return -np.sum(np.multiply(tempTrueClass, logPredictedClass))/len(tempTrueClass)


    def predict(self,inputVal:List[float]):
        # need progress bar in this one too
        for x,y in zip(self.layer_selector[0],inputVal):
            x.value = y
        self.feedforward()
        return self.getPredResult()

    def getPredResult(self):
        return self.layer_selector[-1]

    def __getitem__(self, idx):
        return self.layer_selector[idx]

    def __repr__(self):
        return f"{self.layer_selector}"


    # Procedure, getter, setter or something else

    def regularization(self):
            sum = 0
            if self.regularization_type == "L1":
                for layer in self.layer_selector:
                    abs_val = np.abs(layer)
                    sum += np.sum(abs_val)
            elif self.regularization_type == "L2":
                for layer in self.layer_selector:
                    square_val = np.square(layer)
                    sum += np.sum(square_val)
            self.loss += self.regularization_lambda*sum  # error_accumulate atau loss, idk need to test it

    def feedforward(self):
        self.layer_selector[0].feedforward()

    def batchHelper(self,
                batch_input: np.ndarray,
                batch_class: np.ndarray,
                ):
        self.loss = Neuron(0)
        self.error_acc = 0
        total_samples = len(batch_input)
        i = 0
        while i < total_samples:
            self.loss = Neuron(0)
            batch_end = min(i + self.batch_size, total_samples)
            for j in range(i, batch_end):
                input_sample = list(batch_input[j])
                target_sample = list(batch_class[j])
                self.layer_selector[0].setLayerNeurons(input_sample)
                self.layer_selector[0].feedforward()
                Y_pred = self.layer_selector[-1]._neurons
                Y_target = self.neuronGenerator(np.array, target_sample)
                loss = self.loss_func(Y_pred=Y_pred, Y_target=Y_target)
                self.loss += loss
                self.error_acc += loss
            self.regularization()
            self.loss.backward()
            self.updateWeightHelper()
            i += self.batch_size

    def validate(self,X_val,Y_val):
        loss = 0
        while(len(Y_val)>0):
            X_val_current =  list(X_val[0])
            Y_val_current = list(Y_val[0])
            self.layer_selector[0].setLayerNeurons(X_val_current)
            self.layer_selector[0].feedforward()
            Y_pred = self.layer_selector[-1]._neurons
            Y_target = self.neuronGenerator(np.array,(Y_val_current))
            loss += self.loss_func(Y_pred=Y_pred,Y_target=Y_target)
            X_val=X_val[1:]
            Y_val=Y_val[1:]
        return loss

    def epoch(self,X_train,Y_train,X_val,Y_val):
        X_train = np.array(X_train)
        Y_train = np.array(Y_train)
        X_val = np.array(X_val)
        Y_val = np.array(Y_val)
        train_loss = []
        val_loss = []
        for i in (tqdm(range(self.max_epoch), desc="Epoch") if self.verbose else range(self.max_epoch)):
            # print(f"Epoch {i}")
            self.batchHelper(
                batch_class=Y_train,
                batch_input=X_train
            )
            val = self.validate(X_val=X_val,Y_val=Y_val)
            tl_val = self.error_acc/len(Y_train)
            vl_val = val/len(Y_val)
            train_loss.append(tl_val)
            self.error_acc = Neuron(0)
            val_loss.append(vl_val)

            self.verboseprint("Training Loss:", tl_val, nl=False)
            self.verboseprint("Validation Loss:", vl_val, nl=True)
            # print(train_loss[i],val_loss[i])
        self.history = [train_loss,val_loss]
        return self.history

    def fit(self,X_train,Y_train,X_val,Y_val):
        return self.epoch(X_train,Y_train,X_val,Y_val)

    def updateWeightHelper(self):
        self.layer_selector[0].updateWeightRecursive()

    def visualizeGraph(self,size=(30,15)):
        self.layer_selector[0].visualizeGraph(size=size)

    def visualizeWeightDist(self,indices):
        for i in indices:
            sns.displot(self.layer_selector[i].weight.flatten().astype(float)).set(title=f"Weight Layer-{i}")

    def visualizeWeightGradDist(self,indices):
        for i in indices:
            grad_data = np.array([item.grad for item in self.layer_selector[i].weight.flatten()])
            sns.displot(grad_data.astype(float)).set(title=f"Weight Grad Layer-{i}")

    # Kebodohan: backward untuk grad hanya boleh dipanggil 1x dari 1 titik, kalau kyk
    # gini dipanggil berkali kali dan akhirnya refer ke node yg terakhir di layer
    # terakhir
    # *ada alasan kenapa yg dibackprop cuma errornya doang
    # def backpropagation(self):
    #     backprop = np.vectorize(lambda x: x.backward())
    #     backprop(self.layer_selector[-1]._neurons)
    # Tidak dihapus untuk catatan kebodohan


class Layers:
    # Static Attribute, cuma buat validasi input
    valid_activation_function = {"linear","relu","sigmoid","hyperbolic_tangent","softmax","leaky_relu","swish"}
    valid_weight_initialization = {"zero","uniform","normal"}

    # START OF INITIALIZATION
    def __init__(self,
                 wrapper:LayerWrapper,                      # Wrapper ini buat ngepoint ke wrappernya
                 layer_neuron_width:list,                   # List of neuron width, elemen index ke-n menentukan lebar layer ke-n
                #  activation_function:str = "linear",        # Harus ada di atas, kubuat defaultnya ini, dan rekursif layer selanjutnya sama
                 activation_function:list,                  # activation function untuk setiap layer, elemen idx ke-n merupakan activation function untuk layer ke-n+1 (idx 0 = input_layer -> layer 1)
                 weight_initialization:list,     # Sama kyk activation function
                 current_iter:int = 0,                      # jgn disentuh, ini buat nandain iterasinya, krn inisialisasi rekursif
                 uniform_parameter: tuple = (0,10,0),       # Hanya terpakai kalau activation_functionnya uniform
                 normal_parameter: tuple = (5,3,0),         # kyk di atas, tp normal

                 # Tambah parameter kasih nama, named parameter biar ga broken pemanggilannya
                 ):

        # Cek validasi
        self.inputValidation(
            func_name=activation_function,
            weight_name=weight_initialization,
            layer_neuron_width=layer_neuron_width
            )

        self.wrapper        : LayerWrapper                      = wrapper
        self.current_iter   : int                               = current_iter
        self.neuron_count   : int                               = layer_neuron_width[current_iter]
        self.bias_weight    : Neuron                            = Neuron(0)
        self._neurons       : np.ndarray[any, np.dtype[Neuron]] = self.neuronGenerator(np.zeros,(self.neuron_count))

        if current_iter < len(layer_neuron_width)-1:
            self.weight_initialization  : str    = weight_initialization[current_iter]
        else:
            self.weight_initialization = None

        self.uniform_parameter  : tuple = uniform_parameter
        self.normal_parameter   : tuple = normal_parameter

        if current_iter > 0:
            self.activation_function    : str   = activation_function[current_iter-1]
        else:
            self.activation_function = None

        wrapper.layer_selector.append(self)
        self.next = (
            Layers(
                wrapper=wrapper,
                layer_neuron_width=layer_neuron_width,
                activation_function=activation_function,
                current_iter=current_iter + 1,
                weight_initialization=weight_initialization,
                uniform_parameter=uniform_parameter,
                normal_parameter=normal_parameter
                )
            if current_iter < len(layer_neuron_width) - 1
            else None
        )
        self.initWeight()
    # END OF INITIALIZATION

    # Buat ngeprint
    def __repr__(self):
        return f"{self._neurons}"

    # Kadang reflek Layer[0] dan dapet error, mending lgsg keluarin Neuron nya
    def __getitem__(self, idx):
        return self._neurons[idx]

    def __setitem__(self, idx, val):
        if (isinstance(val,Neuron)):
            self._neurons[idx] = val
        else:
            self._neurons[idx] = Neuron(val)

    # Kalau ada yg aku miss tambahin aja
    def inputValidation(self,
                        func_name:list,
                        weight_name:list,
                        layer_neuron_width:list
                        ):

        if any(el not in Layers.valid_activation_function for el in func_name):
            raise ValueError(f"Valid activation function name: {Layers.valid_activation_function}")

        if any(el  not in Layers.valid_weight_initialization for el in weight_name):
            raise ValueError(f"Valid weight initialization: {Layers.valid_weight_initialization}")

        if len(layer_neuron_width) < 1:
            raise ValueError(f"layer_neuron_width must be a list with at least one positive integer.")

        if not all(isinstance(n, int) and n > 0 for n in layer_neuron_width):
            raise ValueError(f"All elements in layer_neuron_width must be a positive integer")

    # Init weightnya ngikut dari named parameter
    def initWeight(self):
        if self.weight_initialization=="normal":
            mean        = self.normal_parameter[0]
            variance    = self.normal_parameter[1]
            seed        = self.normal_parameter[2]
            self.initWeightNormal(mean,variance,seed)

        elif self.weight_initialization=="uniform":
            low     = self.uniform_parameter[0]
            high    = self.uniform_parameter[1]
            seed    = self.uniform_parameter[2]
            self.initWeightUniform(low,high,seed)

        elif self.weight_initialization=="zero":
            self.initWeightZero()

    # Tiga function di bawah buat initialization, buat manual assign per layer
    def initWeightUniform(
            self,
            low     : float =   0,
            high    : float =   10,
            seed    : int   =   0
            ):

        if self.next != None:
            self.weight_initialization = "uniform"
            rng = np.random.default_rng(seed=seed)
            next_neuron_count = self.next.neuron_count
            self.weight = self.neuronGenerator(rng.uniform,low,high,size=(next_neuron_count,self.neuron_count))

    def initWeightNormal(
            self,
            mean        :   float   =   5,
            variance    :   float   =   3,
            seed        :   int     =   0
            ):
        if self.next != None:
            self.weight_initialization = "normal"
            rng = np.random.default_rng(seed=seed)
            next_neuron_count = self.next.neuron_count
            self.weight = self.neuronGenerator(rng.normal,loc=mean,scale=variance*variance,size=(next_neuron_count,self.neuron_count))

    def initWeightZero(
            self
        ):
        if self.next != None:
            self.weight_initialization = "zero"
            next_neuron_count = self.next.neuron_count
            self.weight = self.neuronGenerator(np.zeros,shape=(next_neuron_count,self.neuron_count))

    # Feedforward rekursif, biar ga tolol kyk sebelumnya, mohon maaf
    def feedforward(self):
        if self.next != None:
            self.next._neurons = np.dot(self._neurons,self.weight.T) + self.bias_weight
            self.next._neurons = self.activate(self.next._neurons)
            self.next.feedforward()

    # Update weight
    def updateWeightRecursive(self):
        if self.next is not None:
            self.weight = Layers.vectorUpdateWeight(self.weight, self.wrapper.learning_rate)
            self.bias_weight = self.bias_weight - (self.wrapper.learning_rate*self.bias_weight.grad)
            for i in self.weight:
                for y in i:
                    y._prev = set()
            self.bias_weight._prev = set()
            self.next.updateWeightRecursive()

    # set Layer ini ngisi manual node nya, baru sadar ga guna jg sih
    # node hidden layer bakal kereplace jg pas feedforward
    def setLayerNeurons(
            self,
            neuron_list:List[float]
            ):
        prev_layer_count = len(self._neurons)
        self._neurons = self.neuronGenerator(np.array,(neuron_list))
        if self.current_iter>0:
            if prev_layer_count != len(self._neurons):
                self.wrapper.layer_selector[self.current_iter-1].initWeight()
        if prev_layer_count!=len(self._neurons):
            self.initWeight()

    # Set lebar/jumlah neuron dari 1 layer
    # di bawah ada set weight karena weight depend on next hidden layer width
    def setLayerWidth(
            self,
            width: int = 1,
        ):
        self._neurons = self.neuronGenerator(np.zeros,(width))
        self.neuron_count = width
        if self.current_iter>0:
            self.wrapper.layer_selector[self.current_iter-1].initWeight()
        self.initWeight()

    def setBias(self, bias:float):
        self.bias_weight = Neuron(bias)

    # Masih sama dgn kode lama, buat fill in neuron di numpy
    def neuronGenerator(self,generator, *args, **kwargs):
        value = generator(*args, **kwargs)
        return np.vectorize(Neuron)(value)

    @staticmethod
    def vectorUpdateWeight(inputVal:Neuron,rate:float):
        def updateSingleNeuron(inputVal:Neuron,rate:float):
            inputVal._prev = set()
            return inputVal-(rate*inputVal.grad)
        updateTemp = np.vectorize(updateSingleNeuron)
        return updateTemp(inputVal,rate)

    # ACTIVATION FUNCTION

    def activate(self,input):
        if self.next.activation_function=="linear":
            return Layers.linear(input)
        elif self.next.activation_function=="relu":
            return Layers.relu(input)
        elif self.next.activation_function=="sigmoid":
            return Layers.sigmoid(input)
        elif self.next.activation_function=="hyperbolic_tangent":
            return Layers.hyperbolic_tangent(input)
        elif self.next.activation_function=="softmax":
            return Layers.soft_max(input)
        elif self.next.activation_function=="swish":
            return Layers.swish(input)

    @staticmethod
    def linear(inputVal):
        return inputVal

    @staticmethod
    def relu(inputVal):
        reluTemp = np.vectorize(lambda x: x if x > 0 else Neuron(0))
        return reluTemp(inputVal)

    # TODO: add parameter, alphanya masih hardcoded 0.5
    def leaky_relu(self, alpha=0.01):
        leakytemp = np.vectorize(lambda x: x if x > 0 else Neuron(0)*Neuron(alpha))
        return leakytemp(self._neurons)

    # TODO: add parameter, beta nya masih hardcoded 1
    @staticmethod
    def swish(inputVal):
        def sigmoidScalar(x:Neuron):
            return x/(1+x.exp(-1))

        vectorized_sigmoid = np.vectorize(sigmoidScalar)
        return vectorized_sigmoid(inputVal)

    @staticmethod
    def sigmoid(inputVal):
        def sigmoidScalar(x:Neuron):
            return 1/(1+x.exp(-1))

        vectorized_sigmoid = np.vectorize(sigmoidScalar)
        return vectorized_sigmoid(inputVal)

    @staticmethod
    def hyperbolic_tangent(inputVal):
        def tanhScalar(x:Neuron):
            return (x.exp()-x.exp(-1))/(x.exp()+x.exp(-1))

        vectorized_tanh = np.vectorize(tanhScalar)
        return vectorized_tanh(inputVal)

    @staticmethod
    def soft_max(inputVal):
        # Softmax output layers will have the same number as the number of output class
        exp = np.vectorize(lambda x: x.exp())
        divide = np.vectorize(lambda x,y: x/y)
        temp = exp(inputVal)
        sumTemp = np.sum(temp)
        return divide(temp,sumTemp)

    def visualizeGraph(self, index=0, G=nx.DiGraph(), layers=[], size=(30, 15)):
        if self.next is not None:
            current_layer_nodes = [f"Layer{index} node {x}" for x in range(len(self._neurons))]
            next_layer_nodes = [f"Layer{index+1} node {x}" for x in range(len(self.next._neurons))]
            current_weights = [[f"{w:.2f}" for w in node] for node in self.weight]
            current_weights_grad = [[f"{w.grad:.2f}" for w in node] for node in self.weight]
            bias_node = f"BiasLayer {index}"
            current_layer_nodes.append(bias_node)
            layers.append(current_layer_nodes)


            for i in range(len(current_layer_nodes)):
                for j in range(len(next_layer_nodes)):
                    if current_layer_nodes[i]==f"BiasLayer {index}":
                        G.add_edge(current_layer_nodes[i],next_layer_nodes[j],
                                   weight=self.bias_weight.value,
                                   grad=self.bias_weight.grad
                                   )
                    else:
                        G.add_edge(current_layer_nodes[i], next_layer_nodes[j],
                                weight=current_weights[j][i],
                                grad=current_weights_grad[j][i])

            self.next.visualizeGraph(index + 1, G, layers, size)

        else:
            last_layer_nodes = [f"Layer{index} node {x}" for x in range(len(self._neurons))]
            layers.append(last_layer_nodes)

            pos = {}
            for layer_idx, layer_nodes in enumerate(layers):
                for i, node in enumerate(layer_nodes):
                    pos[node] = (layer_idx, -i)

            plt.figure(figsize=size)
            nx.draw(G, pos, with_labels=True, node_color='lightgreen',
                    node_size=2000, font_size=14, arrows=True)

            edge_labels = nx.get_edge_attributes(G, 'weight')
            nx.draw_networkx_edge_labels(
                G,
                pos,
                edge_labels=edge_labels,
                font_color='blue',
                label_pos=0.75,
                bbox=None
            )

            edge_labels2 = nx.get_edge_attributes(G, 'grad')
            nx.draw_networkx_edge_labels(
                G,
                pos,
                edge_labels=edge_labels2,
                font_color='red',
                label_pos=0.25,
                bbox=None
            )
            plt.show()

In [ ]:
xorFunc = LayerWrapper(
    activation_function=["sigmoid", "sigmoid", "sigmoid"],
    weight_initialization=["uniform", "uniform", "uniform"],
    loss_function="mse",
    layer_neuron_width=[2,4,4,1],
    batch_size=1,
    learning_rate=0.9,
    max_epoch=4500,
	verbose=1
)


In [ ]:
saved = xorFunc.fit(
    Y_train=[[1],[1],[0],[0]],
    X_train=[[1,0],[0,1],[1,1],[0,0]],
    X_val=[[1,0],[1,1]],
    Y_val=[[1],[0]],
)

In [ ]:
xorFunc.visualizeGraph((12,12))

In [ ]:
x = range(len(xorFunc.history[0]))

plt.plot(x, xorFunc.history[0],label='Train')
plt.plot(x, xorFunc.history[1], label='Val')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=1000, test_size=200)
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = tf.reshape(X_train, [-1, 28, 28, 1])
X_test = tf.reshape(X_test, [-1, 28, 28, 1])

X_train = tf.image.resize(X_train, [14, 14])
X_test = tf.image.resize(X_test, [14, 14])

X_train = tf.reshape(X_train, [X_train.shape[0], -1])
X_test = tf.reshape(X_test, [X_test.shape[0], -1])